# Enviornment 

In [18]:
#! pip install -U --quiet langchain_community tiktoken langchain-mistralai langchainhub chromadb langchain langgraph tavily-python

In [1]:
class Vectorstore:
    data = {"ArithmeticError": "ArithmeticError occurred", "AssertionError": "Assertion error occurred", "LookupError": "Lookup"}
    def invoke(self,text):
        Vectorstore.data.get(text,"ArithmeticError")
    
    def invoke_new(self,text):
        'Yes' if text in Vectorstore.data else 'No'

retriever = Vectorstore()
retrieval_grader = Vectorstore()

In [2]:
module_meta_schema = {'app.py':
                      {'abs_path':'path to file'
                      ,'function_name':{'definition':'definition goes here','description':'','params':[{'type':'dict/list/str/int','data':'data sample if any'},]
                                        ,'returns':{'type':'dict/list/str/int','data':'data sample if any'}
                                        ,'caller_function':'caller function/method name goes here'
                                        }
                      ,'class_name':{'definition':'class definition goes here'
                                     ,'method_name':{'definition':'definition goes here','description':'objective goes here','params':[{'type':'dict/list/str/int','data':'data sample if any'},]
                                                     ,'returns':{'type':'dict/list/str/int','data':'data sample if any'}
                                                     ,'caller_function':'caller function/method name goes here'
                                                     }
                                     },
                      },
                      'car_emi.py':
                       {'abs_path':'path to file'
                        ,'function_name':{'signature':'definition goes here','description':'','params':[{'type':'dict/list/str/int','data':'data sample if any'},]
                                          ,'returns':{'type':'dict/list/str/int','data':'data sample if any'}
                                          ,'caller_function':'caller function/method name goes here'
                                          }
                        ,'class_name':{'signature':'definition goes here'
                                       ,'method_name':{'definition':'definition goes here','description':'objective goes here','params':[{'type':'dict/list/str/int','data':'data sample if any'},]
                                                       ,'returns':{'type':'dict/list/str/int','data':'data sample if any'}
                                                       ,'caller_function':'caller function/method name goes here'
                                                       }
                                     },
                        },
                    }
def dependency_search_tool(text):
    '''text can be module name or function name or module name or class name'''
    return module_meta_schema.find(text)

# Graph 

We build the above workflow as a graph.

## Graph state

The graph `state` contains objects that we want to modify in the graph.

In [3]:
from typing_extensions import TypedDict
from typing import List

class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        dependency_search: whether to add search
        dependency_documents: list of documents 
    """
    question : str
    generation : str
    more_documents : str
    documents : str
    counter : int = 0

In [8]:
from langchain.schema import Document

### Nodes 
def generate(state):
    """
    Generate test case & code

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]
    
    # LLM generation
    generation = "Code generation Node"
    return {"documents": documents, "question": question, "generation": generation}

def more_documents_req(state):
    """
    Determines whether the documents are relevant to generate test cases & code
    If not enough information present in given documents, we will set a flag to run dependency search

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Filtered out irrelevant documents and updated dep_search state
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]
    counter = state["counter"]
    if counter is None:
        counter = 0
    
    # Score each doc
    filtered_docs = []
    more_documents = "No"
    
    score = retrieval_grader.invoke_new('ArithmeticError')
    grade = 'NO'#score.binary_score
    
    if grade.lower() == "yes":
        print("---GRADE: DOCUMENT RELEVANT---")
        documents + "new dependency function goes here"
    else:
        print("---GRADE: DOCUMENT NOT RELEVANT---")
        more_documents = "Yes"
    
    return {"documents": documents, "question": question, "more_documents": more_documents, "counter": counter+1}

### Edges

def decide_to_generate(state):
    """
    Determines whether to generate an answer, or add dependency search

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    question = state["question"]
    more_documents = state["more_documents"]
    documents = state["documents"]
    counter = state['counter']

    if counter >= 5:
        print("---DECISION: GENERATE ENDED with counter exceed---")
        return "generate"
    if more_documents == "Yes":
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print("---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---")
        return "more_documents_req"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"

## Build Graph

This follows the flow we outlined in the figure above.

In [9]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("generate", generate)  # generatae
workflow.add_node("more_documents_req", more_documents_req)  # web search

# Build graph
workflow.set_entry_point("more_documents_req")
workflow.add_edge("more_documents_req", "generate")
workflow.add_conditional_edges(
    "more_documents_req",
    decide_to_generate,
    {
        "more_documents_req": "more_documents_req",
        "generate": "generate",
    },
)
workflow.add_edge("generate", END)

# Compile
app = workflow.compile()

In [10]:
#dir(app)
app.channels

{'question': <langgraph.channels.last_value.LastValue at 0x296451c4e00>,
 'generation': <langgraph.channels.last_value.LastValue at 0x296451c5160>,
 'more_documents': <langgraph.channels.last_value.LastValue at 0x296453a9a30>,
 'documents': <langgraph.channels.last_value.LastValue at 0x296443a4920>,
 'counter': <langgraph.channels.last_value.LastValue at 0x296443a68d0>,
 '__start__': <langgraph.channels.ephemeral_value.EphemeralValue at 0x296451c4da0>,
 'generate': <langgraph.channels.ephemeral_value.EphemeralValue at 0x296453aeff0>,
 'more_documents_req': <langgraph.channels.ephemeral_value.EphemeralValue at 0x296453aca10>,
 'start:more_documents_req': <langgraph.channels.ephemeral_value.EphemeralValue at 0x296453ae060>,
 'branch:more_documents_req:decide_to_generate:more_documents_req': <langgraph.channels.ephemeral_value.EphemeralValue at 0x296453aea80>,
 'branch:more_documents_req:decide_to_generate:generate': <langgraph.channels.ephemeral_value.EphemeralValue at 0x296453aeea0>}

In [11]:
from pprint import pprint
inputs = {"question": "ArithmeticError","documents":"ArithmeticError"}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")
pprint(value["generation"])

---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---
'Finished running: more_documents_req:'
---GENERATE---
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---
'Finished running: generate:'
'Finished running: more_documents_req:'


InvalidUpdateError: Invalid update for channel question with values ['ArithmeticError', 'ArithmeticError']